Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal (i.e. not linear in the number of degrees of freedom). For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [2]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
Draw(shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

Generate a mesh, and perform uniform mesh refinement:

In [3]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)
for l in range(1):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [4]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withedges=False)
print ("we have", fes.ndof, "unknowns")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
f = LinearForm(v*dx)

# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "local")
# c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 177430 unknowns


assemble system and setup preconditioner in parallel:

In [5]:
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [7]:
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=c.mat, printrates=True, maxiter=400)
    gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.024273223974312072     
CG iteration 2, residual = 0.03720447589534199     
CG iteration 3, residual = 0.04829608173279241     
CG iteration 4, residual = 0.04783145202720858     
CG iteration 5, residual = 0.03503614777663428     
CG iteration 6, residual = 0.02937808518603648     
CG iteration 7, residual = 0.031006371846374843     
CG iteration 8, residual = 0.02881189922538989     
CG iteration 9, residual = 0.026991393488043684     
CG iteration 10, residual = 0.02560781323347911     
CG iteration 11, residual = 0.023059227507159427     
CG iteration 12, residual = 0.0201887142527063     
CG iteration 13, residual = 0.017677856690926363     
CG iteration 14, residual = 0.01570390414750895     
CG iteration 15, residual = 0.014508920088754975     
CG iteration 16, residual = 0.013060236654630368     
CG iteration 17, residual = 0.011487172720698016     
CG iteration 18, residual = 0.010359419640115412     
CG iteration 19, residual = 0.00963928071857335

In [9]:
Draw (gfu, draw_vol=True);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

**Exercise:**
* Experiment with differ problem sizes and preconditioners
* How big systems can you solve on your computer (watch out for memory usage)